`The Variable API has been deprecated: Variables are no longer necessary to use autograd with tensors. Autograd automatically supports Tensors with requires_grad set to True. Below please find a quick guide on what has changed:
`
<a>
PyTorch在autograd模块中实现了计算图的相关功能，autograd中的核心数据结构是Variable。Variable封装了tensor，并记录对tensor的操作记录用来构建计算图。Variable的数据结构如图3-2所示，主要包含三个属性：

* data：保存variable所包含的tensor
* grad：保存data对应的梯度，grad也是variable，而不是tensor，它与data形状一致。
* grad_fn： 指向一个Function，记录tensor的操作历史，即它是什么操作的输出，用来构建计算图。如果某一个变量是由用户创建，则它为叶子节点，对应的grad_fn等于None。
</a>

In [8]:
import torch as t

In [7]:
t.ones(3,4)

'torch.FloatTensor'

In [33]:
# 由于我是用的是0.4.0版本的pytorch，所以需要迁移
# 具体看这里 https://blog.csdn.net/Hungryof/article/details/80103264
a = t.ones(3,4,requires_grad=True)
b = t.zeros(3,4)
c = a.add(b)
c

tensor([[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]])

In [36]:
d = c.sum()
d.backward()

In [41]:
# 由用户创建的variable属于叶子节点，对应的grad_fn是None
a.is_leaf, b.is_leaf, c.is_leaf

(True, True, False)

In [40]:
# 此处虽然没有指定c需要求导，但c依赖于a，而a需要求导，
# 因此c的requires_grad属性会自动设为True
a.requires_grad, b.requires_grad, c.requires_grad

(True, False, True)

In [43]:
# c.grad是None, 因c不是叶子节点，它的梯度是用来计算a的梯度
# 所以虽然c.requires_grad = True,但其梯度计算完之后即被释放
c.grad is None

True

<a>计算下面这个函数的导函数： $$
y = x^2\bullet e^x
$$ 它的导函数是： $$
{dy \over dx} = 2x\bullet e^x + x^2 \bullet e^x
$$
    </a>

In [45]:
def f(x):
    y = x**2 * t.exp(x)
    return y

def gradf(x):
    dx = 2*x*t.exp(x) + x**2*t.exp(x)
    return dx


In [46]:
x = t.randn(3,4, requires_grad=True)
y = f(x)
y

tensor([[ 1.4094e-01,  1.8928e-01,  8.5175e-02,  4.4587e-01],
        [ 5.4112e-01,  5.4095e-02,  2.0862e+01,  2.4336e-01],
        [ 6.6231e-04,  7.6863e+00,  1.4614e-01,  6.1493e-03]])

In [47]:
gradf(x)

tensor([[ -0.4508,  -0.4611,   0.7488,  -0.2713],
        [  0.0107,   0.5706,  43.6660,  -0.4519],
        [  0.0528,  18.7757,  -0.4529,   0.1690]])

<font color=red size=4>这里有些不懂为什么要t.ones(y.size())</font>

In [49]:
# autograd的计算结果与利用公式手动计算的结果一致
y.backward(t.ones(y.size()))
x.grad

tensor([[ -0.4508,  -0.4611,   0.7488,  -0.2713],
        [  0.0107,   0.5706,  43.6660,  -0.4519],
        [  0.0528,  18.7757,  -0.4529,   0.1690]])

<a>计算图:反向传播
w -->  
        mul ---> y --->     add ---> z
x -->            b --->
</a>

In [54]:
# 
x = t.ones(1, requires_grad=True)
b = t.rand(1)
w = t.rand(1)
y = 2*x
z = y+b
x.requires_grad,w.requires_grad,y.requires_grad,b.requires_grad,z.requires_grad

(True, False, True, False, True)

In [55]:
# grad_fn可以查看这个variable的反向传播函数，
# z是add函数的输出，所以它的反向传播函数是AddBackward
z.grad_fn

In [57]:
# next_functions保存grad_fn的输入，是一个tuple，tuple的元素也是Function
# 第一个是y，它是乘法(mul)的输出，所以对应的反向传播函数y.grad_fn是MulBackward
# 第二个是b，它是叶子节点，由用户创建，grad_fn为None
z.grad_fn.next_functions

((<MulBackward0 at 0x7f26240ea550>, 0), (None, 0))

<font color=red size=4>这里要注意以下，返回的是tuple，但是tuple中的内容应该是一个list，里面是function</font>

In [73]:
z.grad_fn.next_functions[0][0] == y.grad_fn

True

In [76]:

# 第一个是x，叶子节点，需要求导，梯度是累加的
# 第二个是w，叶子节点，不需要求导，所以为None
y.grad_fn.next_functions

((<AccumulateGrad at 0x7f2624142e10>, 0),)

In [85]:
z.backward()
print(x.grad)

tensor([ 6.])


<a>

PyTorch使用的是动态图，它的计算图在每次前向传播时都是从头开始构建，所以它能够使用Python控制语句（如for、if等）根据需求创建计算图。这点在自然语言处理领域中很有用，它意味着你不需要事先构建所有可能用到的图的路径，图在运行时才构建。
</a>

In [101]:
def abs(x):
    if x[0]>0: return x
    else: return -x
    
x = t.ones(1,requires_grad=True)
y = abs(x)
y.backward()
x.grad

tensor([ 1.])

<font color=red size=4>这里很奇怪的，为什么会输出None,换成Variable就是-1了:
    

</font>
reason: 因为这里x变成一个中间变量了
    相当于tmp = Variable()
    x = -1 * temp

In [114]:
def abs(x):
    if x[0]>0: return x
    else: return -x
    
x = -1 * t.ones(1,requires_grad=True)
y = abs(x)
y.backward()
print(x.grad)

TypeError: new() received an invalid combination of arguments - got (list, requires_grad=bool), but expected one of:
 * (torch.device device)
 * (tuple of ints size, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
 * (torch.Storage storage)
 * (Tensor other)
 * (object data, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad


In [107]:
import torch as t
from torch.autograd import Variable as V
def abs(x):
    if x.data[0]>0: return x
    else: return -x
x = V(t.ones(1),requires_grad=True)
y = abs(x)
y.backward()
x.grad

tensor([ 1.])

In [108]:
x = V(-1*t.ones(1),requires_grad=True)
y = abs(x)
y.backward()
print(x.grad)

tensor([-1.])


<font color=red size=4>
    Solved:
    这里很奇怪的,输出看不懂
    
x1,x2,x3
求偏导，
所以[0 0 0 x2x3,x1x3,x1x2]
</font>

In [136]:


def f(x):
    result = 1
    for ii in x:
        # ii[0] == ii
        if ii>0: result=ii*result 
    return result

x = t.arange(-2,4,requires_grad=True)
y = f(x) # y = x[3]*x[4]*x[5]
print(x)
print(y)
y.backward()
x.grad

tensor([-2., -1.,  0.,  1.,  2.,  3.])
tensor(6.)


tensor([ 0.,  0.,  0.,  6.,  3.,  2.])

<a>


在反向传播过程中非叶子节点的导数计算完之后即被清空。若想查看这些变量的梯度，有两种方法：

    使用autograd.grad函数
    使用hook

autograd.grad和hook方法都是很强大的工具，更详细的用法参考官方api文档，这里举例说明基础的使用。推荐使用hook方法，但是在实际使用中应尽量避免修改grad的值。

</a>

In [140]:
x = t.ones(3,requires_grad=True)
w = t.rand(3,requires_grad=True)
y = x*w
z= y.sum()
x.requires_grad,w.requires_grad,z.requires_grad

(True, True, True)

In [141]:
z.backward()
(x.grad,w.grad,y.grad)

(tensor([ 0.2779,  0.8638,  0.3761]), tensor([ 1.,  1.,  1.]), None)

<a>获取非叶字节点的梯度</a>


In [146]:
# 第一种方法：使用grad获取中间变量的梯度
x = t.ones(3, requires_grad=True)
w = t.rand(3, requires_grad=True)
y = x * w
z = y.sum()
# z对y的梯度，隐式调用backward()
t.autograd.grad(z, y)
# 只是反向传播到了y，并没有到x,w

(tensor([ 1.,  1.,  1.]),)

In [151]:
# 第二种方法：使用hook
# hook是一个函数，输入是梯度，不应该有返回值
def tensor_hook(grad):
    print('y_grad \r\n',grad)
    
x = t.ones(3, requires_grad=True)
w = t.rand(3, requires_grad=True)
y = x * w
hook_handle = y.register_hook(tensor_hook)
z = y.sum()
z.backward()
# 除非你每次都要用hook，否则用完之后记得移除hook
hook_handle.remove()


y_grad 
 tensor([ 1.,  1.,  1.])


<a>
最后再来看看variable中grad属性和backward函数grad_variables参数的含义，这里直接下结论：
    
* variable $\textbf{x}$的梯度是目标函数${f(x)} $对$\textbf{x}$的梯度，$\frac{df(x)}{dx} = (\frac {df(x)}{dx_0},\frac {df(x)}{dx_1},...,\frac {df(x)}{dx_N})$，形状和$\textbf{x}$一致。

* 对于y.backward(grad_variables)中的grad_variables相当于链式求导法则中的$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$中的$\frac{\partial z}{\partial y}$。z是目标函数，一般是一个标量，故而$\frac{\partial z}{\partial y}$的形状与variable $\textbf{y}$的形状一致。z.backward()在一定程度上等价于y.backward(grad_y)。z.backward()省略了grad_variables参数，是因为$z$是一个标量，而$\frac{\partial z}{\partial z} = 1$

   
</a>

In [152]:
x = t.arange(0,3, requires_grad=True)
y = x**2 + x*2
z = y.sum()
z.backward() # 从z开始反向传播
x.grad

tensor([ 2.,  4.,  6.])

In [160]:
x = t.arange(0,3, requires_grad=True)
y = x**2 + x*2
z = y.sum()
print(z)
y_grad_variables = t.Tensor([1,1,1]) # dz/dy
y.backward(y_grad_variables) #从y开始反向传播
x.grad

tensor(11.)


tensor([ 2.,  4.,  6.])

In [ ]:
class Mul(Function):
    
    def forward(ctx, w, x, b, x_requitres_grad =True):
        ctx.x_requires_grad = x_requires_grad
        ctx.save_for_backward(w,x)
        ouput = w * x + b
        return output
    def backward(ctx, grad_output):
        w,x = ctx.saved_variables
        grad